In [78]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, CategoricalArrays
using Plots
using Distributions

In [3]:
const GRB_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-11


In [58]:
preferences_0 = CSV.read("preferences_0.csv", DataFrame);
preferences_1 = CSV.read("preferences_1.csv", DataFrame);

preferences_0 = Matrix(preferences_0)[:,2:end];
preferences_1 = Matrix(preferences_1)[:,2:end];

In [68]:
ones(1,20)' .* preferences_0[1,:] + preferences_1'[1,:]

20×1 Matrix{Float64}:
 -1.0685393044061506
 -0.1532859072887971
 -3.9008022366268795
 -0.6241368115347704
 -1.337433667270756
 -1.5557060669263114
 -2.4567215591712874
  1.386467145515355
 -2.7011399255904704
 -0.7246647545747233
  1.1623497245694752
 -1.0636558173477226
 -0.8183852893007684
  0.14149256001153043
 -1.310275464475294
  0.8768466308140181
 -3.659563983306513
 -0.5835099723665889
 -2.57101173238393
 -0.7933110694996919

In [ ]:
m1 = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables
@variable(m1, z[1:size(preferences_0), 1:20], Bin)

sum(x[:, j])

@objective(m1, Min, sum(-x[i,j] * (preferences_0[i,j] + preferences_1[i,j]) for i in 1:20, j in 1:20))

@constraint(m1, [i=1:20], sum(x[i, :]) == 1)

In [75]:
ones(1,20)' .* preferences_0[1,:] + preferences_1'[1,:]

20×1 Matrix{Float64}:
 -1.0685393044061506
 -0.1532859072887971
 -3.9008022366268795
 -0.6241368115347704
 -1.337433667270756
 -1.5557060669263114
 -2.4567215591712874
  1.386467145515355
 -2.7011399255904704
 -0.7246647545747233
  1.1623497245694752
 -1.0636558173477226
 -0.8183852893007684
  0.14149256001153043
 -1.310275464475294
  0.8768466308140181
 -3.659563983306513
 -0.5835099723665889
 -2.57101173238393
 -0.7933110694996919

In [77]:
rho = 0.5;
# Create a new model

m = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Create variables

#zij = 1 if i and j are matched, 0 otherwise
@variable(m, z[1:size(preferences_0,1), 1:size(preferences_1,1)], Bin)
@variable(m, ui[1:size(preferences_0,1)], Bin)
@variable(m, uj[1:size(preferences_1,1)], Bin)

# Counting variable for unmatched students

#t represents max of mi and mj
#@variable(m, t[1:size(preferences_0,1), 1:size(preferences_1,1)], Bin)

# Create objective

@objective(m, Min, -1*sum(z[i,:]'*(preferences_0[i,:] + preferences_1'[i,:]) for i in 1:size(preferences_0,1)) + rho*(sum(ui) + sum(uj)))

# Create constraints

# Each person can only be matched to one person 
@constraint(m, [i in 1:size(preferences_0,1)], sum(z[i,j] for j in 1:size(preferences_1,1)) <= 1)
@constraint(m, [j in 1:size(preferences_1,1)], sum(z[i,j] for i in 1:size(preferences_0,1)) <= 1)

# If unmatched define ui and uj
@constraint(m, [i in 1:size(preferences_0,1)], ui[i] >= 1 - sum(z[i,j] for j in 1:size(preferences_1,1)))
@constraint(m, [j in 1:size(preferences_1,1)], uj[j] >= 1 - sum(z[i,j] for i in 1:size(preferences_0,1)))

optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 80 rows, 440 columns and 1640 nonzeros
Model fingerprint: 0x00792b20
Variable types: 0 continuous, 440 integer (440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -20.4384452
Presolve removed 0 rows and 92 columns
Presolve time: 0.00s
Presolved: 80 rows, 348 columns, 1272 nonzeros
Variable types: 0 continuous, 348 integer (348 binary)

Root relaxation: objective -4.347703e+01, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     -43.4770278  -43.47703  0.00%     -    0s

Explored 1 nodes (28 simplex iterati

In [79]:
# Get value of zij

zij = value.(z)

20×20 Matrix{Float64}:
  0.0  -0.0   0.0  -0.0   0.0   0.0  …   0.0  -0.0   0.0  -0.0   0.0  -0.0
 -0.0   0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0   0.0   1.0   0.0   0.0
 -0.0  -0.0  -0.0   0.0   0.0   1.0     -0.0   0.0  -0.0  -0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0   0.0  -0.0     -0.0   0.0  -0.0   0.0   1.0   0.0
  0.0  -0.0  -0.0   0.0   0.0   0.0      1.0  -0.0  -0.0   0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0  …   0.0   0.0   0.0  -0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0      0.0   0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   0.0  -0.0   0.0  -0.0     -0.0  -0.0   1.0  -0.0  -0.0  -0.0
  1.0  -0.0  -0.0  -0.0  -0.0  -0.0      0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0   1.0
 -0.0  -0.0   1.0  -0.0  -0.0  -0.0  …   0.0  -0.0   0.0  -0.0  -0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0  -0.0   0.0  -0.0   0.0     -0.0  -0.0  -0.0  -0.0   0.0  -0.0
 -

In [89]:
sum((preferences_0 + preferences_1') .* zij)

20.0